In [1]:
import numpy as np
from sklearn import neural_network

In [2]:
cc = np.load('cluster_centers.npy')
labels = np.load('labels.npy')
tac = np.load('train_art_clusters.npy')

In [4]:
print cc.shape

(200, 50)


In [81]:
print tac.shape

(4154804, 3)


In [6]:
print labels.shape

(233286,)


In [3]:
# relies on chopping off column headers ahead of time. 
artists = np.loadtxt('data/artists.csv', dtype='string', delimiter=',')
print artists
profiles = np.loadtxt('data/profiles_fixed.csv', dtype='string', delimiter=',')
print profiles.shape

# fixing erroneous, irrelevant ages like 1002
max_age = 120
# total = 0
# count = 0
for p in profiles:
    if p[2] != '' and (int(p[2]) >= max_age or int(p[2]) < 0):
        # 22 is the average age of those who put down ages within a reasonable range
        p[2] = '22'

art_dict = {}
count = 0
for a in artists:
    art_dict[a[0]] = count
    count += 1
    
prof_dict = {}
count = 0
for a in profiles:
    prof_dict[a[0]] = count
    count += 1
    
nation_dict = {}
count = 0
for p in profiles:
    if p[3] not in nation_dict:
        nation_dict[p[3]] = None
num_nations = len(nation_dict)
count = 0
for key in nation_dict.keys():
    nation_dict[key] = [0] * num_nations
    nation_dict[key][count] = 1
    count += 1

[['03098741-08b3-4dd7-b3f6-1b0bfa2c879c' 'Liars']
 ['69c4cc43-8163-41c5-ac81-30946d27bb69' 'CunninLynguists']
 ['7a2e6b55-f149-4e74-be6a-30a1b1a387bb' 'The Desert Sessions']
 ..., 
 ['8974da95-e631-45aa-8fd7-aa0c2795f997' 'Harry Gregson-Williams']
 ['8067c102-4996-42bc-9980-06ce2e644eae' 'Soul Coughing']
 ['39c2a93d-9afa-4a22-9bba-c087ab056e1c' 'Jefferson Airplane']]
(233286, 4)


In [4]:
train = np.loadtxt('data/train_fixed.csv', dtype='string', delimiter=',')

In [5]:
def make_one_hot(length, n):
    lst = [0] * length
    lst[n] = 1
    return lst

In [6]:
pre_X = []
pre_Y = []
count = 0
for t in train:
# for t in train[:10]:
    p = prof_dict[t[0]]
    
    temp_x = []
    temp_x += make_one_hot(200, labels[p])
    temp_x += make_one_hot(50, int(tac[count][1]))
    
    if profiles[p][1] == 'f':
        temp_x += make_one_hot(3, 0)
    elif profiles[p][1] == 'm':
        temp_x += make_one_hot(3, 1)
    else:
        temp_x += make_one_hot(3, 2)
        
    if profiles[p][2] == '':
        temp_x += make_one_hot(121, 120)
    else:
        temp_x += make_one_hot(121, int(profiles[p][2]))
        
    temp_x += nation_dict[profiles[p][3]]
        
    pre_X.append(temp_x)
    
    pre_Y.append(int(t[2]))
    
    count += 1

In [7]:
X = np.array(pre_X)
Y = np.array(pre_Y)

In [8]:
def mse(Y, Y_cv):
    diff = Y_cv - Y
    d_sq = np.square(diff)
    mse_res = np.mean(d_sq)
    return mse_res

In [9]:
MLP = neural_network.MLPRegressor(max_iter=200, verbose=True, warm_start=True)
MLP.fit(X[:1000000], Y[:1000000])

Iteration 1, loss = 328105.80899441
Iteration 2, loss = 308562.03931065
Iteration 3, loss = 303659.85609743
Iteration 4, loss = 300933.91086671
Iteration 5, loss = 299122.12108052
Iteration 6, loss = 297755.89428043
Iteration 7, loss = 296716.88577755
Iteration 8, loss = 295879.96557273
Iteration 9, loss = 295178.15150577
Iteration 10, loss = 294573.93928163
Iteration 11, loss = 294067.28546128
Iteration 12, loss = 293627.57244443
Iteration 13, loss = 293212.60136216
Iteration 14, loss = 292830.93635897
Iteration 15, loss = 292490.31754618
Iteration 16, loss = 292179.04589195
Iteration 17, loss = 291864.83342486
Iteration 18, loss = 291556.68891775
Iteration 19, loss = 291299.01866696
Iteration 20, loss = 291033.86165877
Iteration 21, loss = 290756.67134111
Iteration 22, loss = 290520.63023743
Iteration 23, loss = 290288.24987387
Iteration 24, loss = 290047.09293692
Iteration 25, loss = 289808.15908610
Iteration 26, loss = 289574.93208308
Iteration 27, loss = 289364.62011977
Iteration 

/usr/local/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=True)

In [11]:
MLP.fit(X[1000000:2000000], Y[1000000:2000000])

Iteration 201, loss = 211949.00736223
Iteration 202, loss = 210425.20066718
Iteration 203, loss = 209454.44738891
Iteration 204, loss = 208706.21196629
Iteration 205, loss = 208158.10903702
Iteration 206, loss = 207609.17501814
Iteration 207, loss = 207212.39921400
Iteration 208, loss = 206801.14214842
Iteration 209, loss = 206469.93101717
Iteration 210, loss = 206111.08819365
Iteration 211, loss = 205793.73699923
Iteration 212, loss = 205582.97214157
Iteration 213, loss = 205286.68396324
Iteration 214, loss = 205061.07050851
Iteration 215, loss = 204882.43608623
Iteration 216, loss = 204653.18669815
Iteration 217, loss = 204455.29555767
Iteration 218, loss = 204253.28617996
Iteration 219, loss = 204101.48393159
Iteration 220, loss = 203935.08329977
Iteration 221, loss = 203743.99009991
Iteration 222, loss = 203580.31917204
Iteration 223, loss = 203393.72051902
Iteration 224, loss = 203248.81421890
Iteration 225, loss = 203092.00554001
Iteration 226, loss = 202972.67766647
Iteration 22

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=True)

In [16]:
MLP.fit(X[2000000:3000000], Y[2000000:3000000])

Iteration 401, loss = 205975.41594847
Iteration 402, loss = 204630.70116811
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=True)

In [ ]:
Y_cv = MLP.predict(X[-1000:])
print mse(Y[-1000:], Y_cv)

In [17]:
import pickle
pickle.dump(MLP, open("mlp_model_3m.p", "wb"))

In [66]:
MLP = pickle.load('mlp_model.p')

In [ ]:
from sklearn import ensemble
RFR = ensemble.RandomForestRegressor(verbose=1, warm_start=True)
RFR.fit(X[:1000000], Y[:1000000])

In [18]:
test = np.loadtxt('data/test.csv', dtype='string', delimiter=',')

In [19]:
elt_list = []
for t in train:
    elt_list.append(float(t[2]))
elist = sorted(elt_list)
global_median = elist[train.shape[0] / 2]

In [20]:
print tac[0]
def get_artist_labels():
    d = {}
    for i in range(len(tac)):
        if tac[i][1] not in d:
            d[train[i][1]] = int(tac[i][1])
    return d
art_labels = get_artist_labels()

['eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03' '16' '554']


In [21]:
import csv

soln_file = 'data/mlp_3m.csv'

with open('data/test.csv', 'r') as test_fh:
# with open('data/train.csv', 'r') as test_fh:
    test_reader = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_reader, None)
    
#     print test.shape

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])
        
        count = 0
        for row in test_reader:
            id     = row[0]
            user   = row[1]
            artist = row[2]
            
            if artist in art_labels and user in prof_dict:
                    p = prof_dict[user]

                    temp_x = []
                    temp_x += make_one_hot(200, labels[p])
                    temp_x += make_one_hot(50, art_labels[artist])

                    if profiles[p][1] == 'f':
                        temp_x += make_one_hot(3, 0)
                    elif profiles[p][1] == 'm':
                        temp_x += make_one_hot(3, 1)
                    else:
                        temp_x += make_one_hot(3, 2)

                    if profiles[p][2] == '':
                        temp_x += make_one_hot(121, 120)
                    else:
                        temp_x += make_one_hot(121, int(profiles[p][2]))

                    temp_x += nation_dict[profiles[p][3]]

                    x = np.array([temp_x])

                    pred = MLP.predict(x)[0]
                    if pred > 0:
                        soln_csv.writerow([id, MLP.predict(x)[0]])
                    else:
                        soln_csv.writerow([id, 0])
            else:
                soln_csv.writerow([id, global_median + 120.0])
                
            if count % 10000 == 0:
                print count,
            count += 1

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 480000 490000 500000 510000 520000 530000 540000 550000 560000 570000 580000 590000 600000 610000 620000 630000 640000 650000 660000 670000 680000 690000 700000 710000 720000 730000 740000 750000 760000 770000 780000 790000 800000 810000 820000 830000 840000 850000 860000 870000 880000 890000 900000 910000 920000 930000 940000 950000 960000 970000 980000 990000 1000000 1010000 1020000 1030000 1040000 1050000 1060000 1070000 1080000 1090000 1100000 1110000 1120000 1130000 1140000 1150000 1160000 1170000 1180000 1190000 1200000 1210000 1220000 1230000 1240000 1250000 1260000 1270000 1280000 1290000 1300000 1310000 1320000 1330000 1340000 1350000 1360000 1370000 1380000 13